In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

In [2]:
def problem_2(df, k=5, t=0.2):
    list = []
    # write your logic here, test is a dataframe
    test = 0

    train_val, test = train_test_split(df,test_size=t, train_size=1-t)
    list = np.array_split(train_val, k)

    return list, test

In [28]:
# Problem 3
def problem_3(filename, column_label):
    # write your logic here, df is a dataframe instead
    df = pd.read_csv(filename)
    dummies = pd.get_dummies(df[column_label], dtype=int)
    df=pd.concat([df,dummies],axis=1)
    return df

In [32]:
# Problem 4
def problem_4(path):
    # Don't touch the settings below
    desired_length, n_fft, hop_len, n_mels, top_db = 100000, 1024, 256, 32, 80
    list_of_mel_spec_db = []
    # retrieve the list of files under path
    file_names = os.listdir(path)

    for file in file_names:
        # write your logic here: load audio file
        full_path = os.path.join(path, file)
        waveform, sample_rate = torchaudio.load(full_path)

        # get the current length of the waveform
        current_length = waveform.size(1)  # Size (channels, length)

        # write your logic here:
        # truncate the waveform, or pad the waveform with zeros
        if current_length > desired_length:
            waveform = waveform[:,:desired_length]
        else:
            zeros = torch.zeros(waveform.size(0), desired_length - current_length)
            waveform = torch.cat([waveform, zeros], dim=1)

        # write your logic here: define mel spec with the settings
        mel_transform = transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            hop_length=hop_len,
            n_mels=n_mels,
        )

        # write your logic here: apply the transform to the waveform
        mel_spec = mel_transform(waveform)

        # write your logic here: convert to decibels with the settings
        mel_spec_db = transforms.AmplitudeToDB(top_db=top_db)(mel_spec)

        list_of_mel_spec_db.append(mel_spec_db)

    # convert the list to a tensor
    # assume all spec have the same shape
    mel_spec_tensor = torch.stack(list_of_mel_spec_db)

    return mel_spec_tensor

In [33]:
tensor = problem_4("TRAIN")
print("Type of tensor:", type(tensor))
print("Shape of tensor:", tensor.shape)
torch.save(tensor, "problem_4.pt")

ImportError: TorchCodec is required for load_with_torchcodec. Please install torchcodec to use this function.